# Black 76 Commodity Options Pricing Example

This notebook demonstrates output from a toy analytics implementation, it can be run after market data load.

Here we use pricing & data endpoints available on the client proxy RestClient API - a server side REST service delegates these requests to the LocalClient API.

Both RestClient & LocalClient share the same interface, so LocalClient enables local development:

`from api import RestClient as c`

`from api import LocalClient as c`

In [1]:
import datetime
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from api import RestClient as c

***** Shell Trading API *****
TRADE_DATE = 2022-12-09
MARKET_DATE = 2022-12-09
ROOT_PATH = C:\Users\magicmonk\PycharmProjects\shell
MARKET_DATA_PATH = C:\Users\magicmonk\PycharmProjects\shell\data
REST_API_URL = http://127.0.0.1:5000


We can easily view the available endpoints using


`c.endpoints()` and inspect each endpoint easily e.g. `c.commodity_option_price?`

In [2]:
c.endpoints()

['commodity_option_greeks',
 'commodity_option_price',
 'data',
 'endpoints',
 'env_variables',
 'option_greeks',
 'option_price',
 'save',
 'symbols',
 'tweak_env']

Using `c.commodity_option_price?`, we can view the endpoint implementation details:

Signature:
c.commodity_option_price(
    contract: str,
    month: str,
    year: str,
    option_type: str,
    strike: str,
    exchange_code: str = None,
    lot_price: bool = False,
) -> float

Docstring:
Black 76 option pricer endpoint

Implied volatility is calculated using the OptionWorks Futures Implied Volatility Model (IVM):
- This assumes a volatility skew curve using a 6-degree polynomial model
- IV = AtM + Beta1*x + Beta2*x^2 + Beta3*x^3 + Beta4*x^4 + Beta5*x^5 + Beta6*x^6 (where x is moneyness)
- https://data.nasdaq.com/data/OWF-optionworks-futures-options/documentation

Time to expiry is calculated using market data option calenders for the specific Option contract code

Cost of carry is set to zero

Risk free rate is weighted interpolation using Fed Reserver daily Treasury Contant Maturity Rates
- https://support.carta.com/s/article/black-scholes

:param contract: Options contract e.g. 'BRENT', 'WTI', 'HH' etc
:param month: Expiry month e.g. 'JAN', 'FEB', 'MAR', etc
:param year: Expiry year e.g. '2025'
:param option_type: Use either 'c' (call) or 'p' (put)
:param strike: Optional, defaults to ATM if not set
:param exchange_code: Optional, e.g. 'ICE', 'NYM' etc
:param lot_price: Optional, set to true to return the Option price * Futures lot size
:return: Option price/premium
File:      c:\users\magicmonk\pycharmprojects\shell\api.py
Type:      function

In [3]:
# BRENT Jan25 Call ATM

c.commodity_option_price(
    contract='BRENT',
    exchange_code='ICE', # Can be removed to allow use of system default exchange code
    month='JAN',
    year='2025',
    option_type='C',
    lot_price=True # Scale the option price by the lot size (maintained as reference data per contract)
)

16386.30042160861

In [4]:
# BRENT Jan25 Call Strike 100 USD/BBL
c.commodity_option_price(
    contract='BRENT',
    exchange_code='ICE', # Can be removed to allow use of system default exchange code
    month='JAN',
    year='2025',
    option_type='C',
    strike=100,
    lot_price=False # By default this is set to False, so can omit
)

8.652043311547885

Greeks are accessed using the `commodity_option_greeks` endpoint, here we omit both exchange code (default is used) and strike (ATM is assumed)

In [5]:
c.commodity_option_greeks(
    contract='BRENT',
    month='JAN',
    year='2025',
    option_type='C'
)

{'DELTA': 0.5640041477987826,
 'GAMMA': 0.007775077334308039,
 'THETA': -3.319508232250979,
 'VEGA': 39.0449384605144,
 'RHO': 55.62303880431869}

In [6]:
# HH Mar24 Put Strike 10 USD/MMBTu
c.commodity_option_price(
    contract='HH',
    month='MAR',
    year='2024',
    option_type='P',
    strike=10
)

5.013212970040467

The Black 76 model extends the Generalised Black Scholes (GBS) model implementation, where we use GBS to benchmark test

In [7]:
"""
Benchmark test:
Black76('c', fs=19, x=19, t=0.75, r=0.10, v=0.28) = 1.70105072524
"""
c.option_price(
    option_type='C', x=19, fs=19, t=0.75, b=0., r=0.1, v=0.28
)

1.7010507252362679

In [8]:
c.option_greeks(
    option_type='C', x=19, fs=19, t=0.75, b=0., r=0.1, v=0.28
)

{'DELTA': 0.5086362359336519,
 'GAMMA': 0.07974503467912114,
 'THETA': -0.9583828622275525,
 'VEGA': 6.045471079024174,
 'RHO': 5.972278318127339}

We can use the `env_variables` endpoint to view available system settings

The `tweak_env` endpoint allows tweaking for the environment `TRADE_DATE` to allow for time travel

In [9]:
c.env_variables()

{'MARKET_DATA_PATH': 'C:\\Users\\magicmonk\\PycharmProjects\\shell\\data',
 'MARKET_DATE': 'Fri, 09 Dec 2022 00:00:00 GMT',
 'REST_API_URL': 'http://127.0.0.1:5000',
 'ROOT_PATH': 'C:\\Users\\magicmonk\\PycharmProjects\\shell',
 'TRADE_DATE': 'Thu, 01 Dec 2022 00:00:00 GMT'}

In [10]:
import datetime as dt

c.tweak_env('TRADE_DATE', dt.date(day=1, month=12, year=2022))

'Env tweaked successful for key=TRADE_DATE, value=2022-12-01'

In [11]:
c.env_variables()

{'MARKET_DATA_PATH': 'C:\\Users\\magicmonk\\PycharmProjects\\shell\\data',
 'MARKET_DATE': 'Fri, 09 Dec 2022 00:00:00 GMT',
 'REST_API_URL': 'http://127.0.0.1:5000',
 'ROOT_PATH': 'C:\\Users\\magicmonk\\PycharmProjects\\shell',
 'TRADE_DATE': 'Thu, 01 Dec 2022 00:00:00 GMT'}

In [12]:
# BRENT Jan25 Call Strike 100 USD/BBL
c.commodity_option_price(
    contract='BRENT',
    exchange_code='ICE', # Can be removed to allow use of system default exchange code
    month='JAN',
    year='2025',
    option_type='C',
    strike=100,
    lot_price=False # By default this is set to False, so can omit
)

8.652043311547885